In [1]:
import pandas as pd
import numpy as np
import os

## Results from the paper

In [2]:
GRU = {
    'mort_hosp':{'auc': 0.8504, 'auprc': 0.5215, 'F1': 0.4229},
    'mort_icu': {'auc': 0.8632, 'auprc': 0.4651, 'F1': 0.3630},
    'los_3':    {'auc': 0.6740, 'auprc': 0.6017, 'F1': 0.5336},
    'los_7':    {'auc': 0.7054, 'auprc': 0.1625, 'F1': 0.0233},
}

# Averaged Multimodal

word2vec_avg = {
    'mort_hosp':{'auc': 0.8642, 'auprc': 0.5422, 'F1': 0.4542},
    'mort_icu': {'auc': 0.8717, 'auprc': 0.4847, 'F1': 0.4230},
    'los_3':    {'auc': 0.6863, 'auprc': 0.6181, 'F1': 0.5419},
    'los_7':    {'auc': 0.7159, 'auprc': 0.1791, 'F1': 0.0135},
}

fastText_avg = {
    'mort_hosp':{'auc': 0.8609, 'auprc': 0.5447, 'F1': 0.4550},
    'mort_icu': {'auc': 0.8714, 'auprc': 0.4836, 'F1': 0.4291},
    'los_3':    {'auc': 0.6855, 'auprc': 0.6159, 'F1': 0.5446},
    'los_7':    {'auc': 0.7131, 'auprc': 0.1757, 'F1': 0.0102},
}

concat_avg = {
    'mort_hosp':{'auc': 0.8598, 'auprc': 0.5419, 'F1': 0.4566},
    'mort_icu': {'auc': 0.8690, 'auprc': 0.4828, 'F1': 0.4076},
    'los_3':    {'auc': 0.6861, 'auprc': 0.6169, 'F1': 0.5470},
    'los_7':    {'auc': 0.7159, 'auprc': 0.1767, 'F1': 0.0137},
}


# Proposed Model

word2vec_proposed = {
    'mort_hosp':{'auc': 0.8755, 'auprc': 0.5587, 'F1': 0.4723},
    'mort_icu': {'auc': 0.8835, 'auprc': 0.4923, 'F1': 0.4302},
    'los_3':    {'auc': 0.6954, 'auprc': 0.6268, 'F1': 0.5504},
    'los_7':    {'auc': 0.7255, 'auprc': 0.1878, 'F1': 0.0158},
}

fastText_proposed = {
    'mort_hosp':{'auc': 0.8715, 'auprc': 0.5568, 'F1': 0.4687},
    'mort_icu': {'auc': 0.8785, 'auprc': 0.4878, 'F1': 0.4309},
    'los_3':    {'auc': 0.6961, 'auprc': 0.6255, 'F1': 0.5587},
    'los_7':    {'auc': 0.7181, 'auprc': 0.1801, 'F1': 0.0108},
}

concat_proposed = {
    'mort_hosp':{'auc': 0.8698, 'auprc': 0.5535, 'F1': 0.4638},
    'mort_icu': {'auc': 0.8766, 'auprc': 0.4874, 'F1': 0.4224},
    'los_3':    {'auc': 0.6993, 'auprc': 0.6277, 'F1': 0.5582},
    'los_7':    {'auc': 0.7192, 'auprc': 0.1825, 'F1': 0.0138},
}


Place them into pandas df to be used later

In [3]:
# paper_results_07
paper_results_07 = pd.DataFrame.from_dict(GRU, orient='index')
paper_results_07 = paper_results_07.reset_index()
paper_results_07.rename(columns={'index':'task'}, inplace=True)
paper_results_07['layer'] = 'GRU'

# paper_results_08
word2vec_avg_pd = pd.DataFrame.from_dict(word2vec_avg, orient='index')
word2vec_avg_pd = word2vec_avg_pd.reset_index()
word2vec_avg_pd.rename(columns={'index':'task'}, inplace=True)
word2vec_avg_pd['embedding'] = 'word2vec'

fastText_avg_pd = pd.DataFrame.from_dict(fastText_avg, orient='index')
fastText_avg_pd = fastText_avg_pd.reset_index()
fastText_avg_pd.rename(columns={'index':'task'}, inplace=True)
fastText_avg_pd['embedding'] = 'fasttext'

concat_avg = pd.DataFrame.from_dict(word2vec_avg, orient='index')
concat_avg = concat_avg.reset_index()
concat_avg.rename(columns={'index':'task'}, inplace=True)
concat_avg['embedding'] = 'concat'

paper_results_08 = pd.concat([word2vec_avg_pd, fastText_avg_pd, concat_avg])

# paper_results_09
word2vec_proposed_pd = pd.DataFrame.from_dict(word2vec_proposed, orient='index')
word2vec_proposed_pd = word2vec_proposed_pd.reset_index()
word2vec_proposed_pd.rename(columns={'index':'task'}, inplace=True)
word2vec_proposed_pd['embedding'] = 'word2vec'

fastText_proposed_pd = pd.DataFrame.from_dict(fastText_proposed, orient='index')
fastText_proposed_pd = fastText_proposed_pd.reset_index()
fastText_proposed_pd.rename(columns={'index':'task'}, inplace=True)
fastText_proposed_pd['embedding'] = 'fasttext'

concat_proposed_pd = pd.DataFrame.from_dict(concat_proposed, orient='index')
concat_proposed_pd = concat_proposed_pd.reset_index()
concat_proposed_pd.rename(columns={'index':'task'}, inplace=True)
concat_proposed_pd['embedding'] = 'concat'

paper_results_09 = pd.concat([word2vec_avg_pd, fastText_avg_pd, concat_avg])


## Functions to produce tabular outputs

In [4]:
# Add n blank rows
def add_blank_rows_df(df, after_each_n):
    new_index = pd.RangeIndex(len(df) + np.ceil(len(df)/after_each_n))
    new_df = pd.DataFrame('', index=new_index, columns=df.columns)
    
    ids = []
    jump = 0
    for i in range(0, len(df)):
        ids.append(i + np.floor(i/after_each_n))
        
    new_df.loc[ids] = df.values
    return new_df

def get_results(result_path: str,
                paper_results,
                model_category: str,
                sort_by:str,
                group_by: list,
                exp_naming: list,
                var_type: str):
    
    result_dists = []
    exps = os.listdir(result_path)
    
    # Pulls the results from the `.p` files 
    for exp in exps:
        if '.p' not in exp:
            continue
        result_dict = pd.read_pickle(os.path.join(result_path, exp))
        result_dict[exp_naming[0]], result_dict[exp_naming[1]], result_dict[exp_naming[2]], result_dict[exp_naming[3]] = exp.split("-")[:4]
        result_dict["model_category"] = model_category
        result_dists.append(result_dict)
        
    result = pd.DataFrame(result_dists)
    result.sort_values(by=['task', sort_by], inplace=True, ascending=False)

    # Generate the summary results table
    summary_results = get_summary_results(result, 
                                          model_category,
                                          sort_by,
                                          group_by,
                                          exp_naming,
                                          var_type)
    
    
    results_difference = get_results_difference(result,
                                                paper_results,
                                                model_category,
                                                sort_by,
                                                group_by,
                                                exp_naming,
                                                var_type)
    
    return result, summary_results, results_difference

def get_summary_results(result, 
                        model_category: str,
                        sort_by:str,
                        group_by: list,
                        exp_naming: list,
                        var_type: str):
    
    summary_results = result.groupby(by=group_by).agg({"auc":['mean','std'], "auprc":['mean','std'], "F1":['mean','std'], 'acc':['mean','std']})
    summary_results.columns = summary_results.columns.map('_'.join).str.strip('_')
    summary_results.reset_index(inplace=True)
    summary_results["model_category"] = model_category
    
    # Get values in terms of %
    for col in ['auc_mean', 'auc_std', 'auprc_mean', 'auprc_std',
       'F1_mean', 'F1_std', 'acc_mean', 'acc_std']:
       summary_results[col] = summary_results[col] * 100
    
    # Rename the tasks
    task_mapping = {
        "los_3": "LOS > 3 Days",
        "los_7": "LOS > 7 Days",
        "mort_hosp": "In-Hospital Mortality",
        "mort_icu": "In-ICU Mortality"
    }
    for k, v in task_mapping.items():
        summary_results.loc[summary_results["task"] == k, "task"] = v
    
    # Format the numeric results
    for matric in ["auc", "auprc", "F1", "acc"]:
        summary_results[matric] = summary_results[[f"{matric}_mean", f"{matric}_std"]].apply(lambda x: "{:.2f} +/- {:.3f}".format(x[0], x[1]), axis=1)
    
    # Order the results in the manner desired
    summary_results = summary_results[["task", "model_category",var_type, "auc", "auprc", "F1"]]

    summary_results.sort_values(by=[sort_by], inplace=True, ascending=False)
    summary_results.sort_values(by=['task'], inplace=True, ascending=True)

    return summary_results


def get_results_difference(result, 
                           paper_results,
                           model_category: str,
                           sort_by:str,
                           group_by: list,
                           exp_naming: list,
                           var_type: str):
    
    results_difference = result.groupby(by=group_by).agg({"auc":['mean'], "auprc":['mean'], "F1":['mean']})
    results_difference = results_difference.rename(columns={'mean':''})
    results_difference.columns = results_difference.columns.map('_'.join).str.strip('_')
    results_difference.reset_index(inplace=True)
    results_difference["model_category"] = model_category
    
    
    # Take the difference between the reported results in the paper and what we have recorded [auc, auprc, F1]
    results_difference = results_difference.assign(auc = results_difference['auc'] - results_difference.merge(paper_results, 
                                                                                         on=['task',var_type], 
                                                                                         suffixes=('','_y'), 
                                                                                         how='left')['auc_y'])  
    results_difference = results_difference.assign(auprc = results_difference['auprc'] - results_difference.merge(paper_results, 
                                                                                         on=['task',var_type], 
                                                                                         suffixes=('','_y'), 
                                                                                         how='left')['auprc_y']) 
    results_difference = results_difference.assign(F1 = results_difference['F1'] - results_difference.merge(paper_results, 
                                                                                         on=['task',var_type], 
                                                                                         suffixes=('','_y'), 
                                                                                         how='left')['F1_y']) 
    
    results_difference.loc[results_difference.merge(paper_results, 
                                                     on=['task',var_type], 
                                                     suffixes=('','_y'), 
                                                     how='left')['auc_y'].isnull(), 'auc'] = '-'
    results_difference.loc[results_difference.merge(paper_results, 
                                                     on=['task',var_type], 
                                                     suffixes=('','_y'), 
                                                     how='left')['auprc_y'].isnull(), 'auprc'] = '-'
    results_difference.loc[results_difference.merge(paper_results, 
                                                     on=['task',var_type], 
                                                     suffixes=('','_y'), 
                                                     how='left')['F1_y'].isnull(), 'F1'] = '-'


    # Get values in terms of %
    for col in ['auc', 'auprc', 'F1']:
       results_difference[col] = results_difference[col] * 100
    
    # Rename the tasks
    task_mapping = {
        "los_3": "LOS > 3 Days",
        "los_7": "LOS > 7 Days",
        "mort_hosp": "In-Hospital Mortality",
        "mort_icu": "In-ICU Mortality"
    }
    for k, v in task_mapping.items():
        results_difference.loc[results_difference["task"] == k, "task"] = v
    
    # Format the numeric results
    
    for metric in ["auc", "auprc", "F1"]:
        results_difference[metric] = results_difference[[metric]].apply(lambda x: "{:.2f}".format(x[0]) if not isinstance(x[0], str) else '-', axis=1)
    
    # Order the results in the manner desired
    results_difference = results_difference[["task", "model_category",var_type, "auc", "auprc", "F1"]]
    
    results_difference.sort_values(by=[sort_by], inplace=True, ascending=False)
    results_difference.sort_values(by=['task'], inplace=True, ascending=True)
    
    return results_difference


In [5]:
paper_results_08

,task,auc,auprc,F1,embedding
0,mort_hosp,0.8642,0.5422,0.4542,word2vec
1,mort_icu,0.8717,0.4847,0.4230,word2vec
2,los_3,0.6863,0.6181,0.5419,word2vec
3,los_7,0.7159,0.1791,0.0135,word2vec
0,mort_hosp,0.8609,0.5447,0.4550,fasttext
1,mort_icu,0.8714,0.4836,0.4291,fasttext
2,los_3,0.6855,0.6159,0.5446,fasttext
3,los_7,0.7131,0.1757,0.0102,fasttext
0,mort_hosp,0.8642,0.5422,0.4542,concat
1,mort_icu,0.8717,0.4847,0.4230,concat


# Only time-series data results

In [6]:
results_07, summary_results_07, results_difference_07 = get_results(
    result_path="results/07-baseline/GRU/hidden_units_256",
    paper_results=paper_results_07,
    model_category="GRU",
    sort_by="auc",
    group_by=["task", "layer"],
    exp_naming=["hidden_units","layer","task","iteration"],
    var_type="layer")

summary_results_07

,task,model_category,layer,auc,auprc,F1
2,In-Hospital Mortality,GRU,GRU,87.58 +/- 0.288,55.26 +/- 0.975,43.93 +/- 2.186
3,In-ICU Mortality,GRU,GRU,88.52 +/- 0.451,51.29 +/- 1.173,42.62 +/- 2.296
0,LOS > 3 Days,GRU,GRU,69.49 +/- 0.511,63.85 +/- 0.562,54.95 +/- 1.255
1,LOS > 7 Days,GRU,GRU,73.72 +/- 0.664,21.23 +/- 0.787,4.58 +/- 1.901


In [7]:
results_difference_07

,task,model_category,layer,auc,auprc,F1
2,In-Hospital Mortality,GRU,GRU,2.54,3.11,1.64
3,In-ICU Mortality,GRU,GRU,2.20,4.78,6.32
0,LOS > 3 Days,GRU,GRU,2.09,3.68,1.59
1,LOS > 7 Days,GRU,GRU,3.18,4.98,2.25


# Baseline time-series and NLP data results

In [8]:
results_08, summary_results_08, results_difference_08 = get_results(
    result_path="results/08-multimodal",
    paper_results=paper_results_08,
    model_category="Average Multimodal",
    sort_by="auc",
    group_by=["task", "embedding"],
    exp_naming=["layer","hidden_units","embedding","task"],
    var_type="embedding")

summary_results_08_print = add_blank_rows_df(summary_results_08, 4)
summary_results_08_print

,task,model_category,embedding,auc,auprc,F1
0,In-Hospital Mortality,Average Multimodal,clinicalBERT,88.35 +/- 0.104,58.10 +/- 0.404,44.51 +/- 1.519
1,In-Hospital Mortality,Average Multimodal,word2vec,88.26 +/- 0.289,58.91 +/- 0.679,47.24 +/- 2.245
2,In-Hospital Mortality,Average Multimodal,concat,88.21 +/- 0.195,58.62 +/- 0.519,48.02 +/- 1.286
3,In-Hospital Mortality,Average Multimodal,fasttext,87.74 +/- 0.323,57.61 +/- 0.451,46.84 +/- 1.530
4,,,,,,
5,In-ICU Mortality,Average Multimodal,word2vec,89.17 +/- 0.176,53.55 +/- 0.414,46.13 +/- 1.631
6,In-ICU Mortality,Average Multimodal,clinicalBERT,89.14 +/- 0.146,53.01 +/- 0.402,43.97 +/- 2.298
7,In-ICU Mortality,Average Multimodal,concat,89.05 +/- 0.252,53.16 +/- 0.372,45.48 +/- 1.420
8,In-ICU Mortality,Average Multimodal,fasttext,88.89 +/- 0.278,52.05 +/- 0.613,44.89 +/- 1.829
9,,,,,,


In [9]:
results_difference_08_print = add_blank_rows_df(results_difference_08, 4)
results_difference_08_print

,task,model_category,embedding,auc,auprc,F1
0,In-Hospital Mortality,Average Multimodal,word2vec,1.84,4.69,1.82
1,In-Hospital Mortality,Average Multimodal,concat,1.79,4.40,2.60
2,In-Hospital Mortality,Average Multimodal,fasttext,1.65,3.14,1.34
3,In-Hospital Mortality,Average Multimodal,clinicalBERT,-,-,-
4,,,,,,
5,In-ICU Mortality,Average Multimodal,word2vec,2.00,5.08,3.83
6,In-ICU Mortality,Average Multimodal,concat,1.88,4.69,3.18
7,In-ICU Mortality,Average Multimodal,fasttext,1.75,3.69,1.98
8,In-ICU Mortality,Average Multimodal,clinicalBERT,-,-,-
9,,,,,,


# Proposed models

In [10]:
results_09, summary_results_09, results_difference_09 = get_results(
    result_path="results/09-cnn",
    paper_results=paper_results_09,
    model_category="Proposed Model",
    sort_by="embedding",
    group_by=["task", "embedding"],
    exp_naming=["sequence_name","hidden_unit_size","embedding","task"],
    var_type="embedding")

summary_results_09_print = add_blank_rows_df(summary_results_09, 3)
summary_results_09_print

,task,model_category,embedding,auc,auprc,F1
0,In-Hospital Mortality,Proposed Model,word2vec,88.00 +/- 0.187,58.06 +/- 0.559,45.53 +/- 1.879
1,In-Hospital Mortality,Proposed Model,fasttext,87.89 +/- 0.329,57.24 +/- 0.646,44.49 +/- 1.630
2,In-Hospital Mortality,Proposed Model,concat,88.14 +/- 0.170,57.68 +/- 0.386,45.69 +/- 1.620
3,,,,,,
4,In-ICU Mortality,Proposed Model,word2vec,88.70 +/- 0.360,52.18 +/- 0.687,43.09 +/- 2.212
5,In-ICU Mortality,Proposed Model,fasttext,88.54 +/- 0.253,51.71 +/- 0.353,43.29 +/- 1.560
6,In-ICU Mortality,Proposed Model,concat,88.72 +/- 0.192,52.19 +/- 0.753,42.59 +/- 2.028
7,,,,,,
8,LOS > 3 Days,Proposed Model,word2vec,70.43 +/- 0.169,64.40 +/- 0.329,55.82 +/- 1.042
9,LOS > 3 Days,Proposed Model,fasttext,69.71 +/- 0.132,63.65 +/- 0.340,54.90 +/- 1.391


In [11]:
results_difference_09_print = add_blank_rows_df(results_difference_09, 3)
results_difference_09_print

,task,model_category,embedding,auc,auprc,F1
0,In-Hospital Mortality,Proposed Model,word2vec,1.58,3.84,0.11
1,In-Hospital Mortality,Proposed Model,fasttext,1.80,2.77,-1.01
2,In-Hospital Mortality,Proposed Model,concat,1.72,3.46,0.27
3,,,,,,
4,In-ICU Mortality,Proposed Model,word2vec,1.53,3.71,0.79
5,In-ICU Mortality,Proposed Model,fasttext,1.40,3.35,0.38
6,In-ICU Mortality,Proposed Model,concat,1.55,3.72,0.29
7,,,,,,
8,LOS > 3 Days,Proposed Model,word2vec,1.80,2.59,1.63
9,LOS > 3 Days,Proposed Model,fasttext,1.16,2.06,0.44


# Make same as paper (Table 3)

In [12]:
def table_3(df1, df2):

    table_3_df = pd.concat([df1, df2])
    table_3_df = table_3_df.drop(columns=['layer']).fillna('')
    table_3_df = table_3_df[["task", "model_category",'embedding', "auc", "auprc", "F1"]]
    table_3_df.sort_values(by='task', inplace=True, ascending=True)
    table_3_df = add_blank_rows_df(table_3_df, 4)
    return table_3_df

summary_results_paper_1 = table_3(summary_results_07, summary_results_08)
summary_results_paper_1

,task,model_category,embedding,auc,auprc,F1
0,In-Hospital Mortality,GRU,,87.58 +/- 0.288,55.26 +/- 0.975,43.93 +/- 2.186
1,In-Hospital Mortality,Average Multimodal,clinicalBERT,88.35 +/- 0.104,58.10 +/- 0.404,44.51 +/- 1.519
2,In-Hospital Mortality,Average Multimodal,word2vec,88.26 +/- 0.289,58.91 +/- 0.679,47.24 +/- 2.245
3,In-Hospital Mortality,Average Multimodal,concat,88.21 +/- 0.195,58.62 +/- 0.519,48.02 +/- 1.286
4,,,,,,
5,In-Hospital Mortality,Average Multimodal,fasttext,87.74 +/- 0.323,57.61 +/- 0.451,46.84 +/- 1.530
6,In-ICU Mortality,Average Multimodal,fasttext,88.89 +/- 0.278,52.05 +/- 0.613,44.89 +/- 1.829
7,In-ICU Mortality,Average Multimodal,concat,89.05 +/- 0.252,53.16 +/- 0.372,45.48 +/- 1.420
8,In-ICU Mortality,Average Multimodal,word2vec,89.17 +/- 0.176,53.55 +/- 0.414,46.13 +/- 1.631
9,,,,,,


In [13]:
results_difference_paper_1 = table_3(results_difference_07, results_difference_08)
results_difference_paper_1

,task,model_category,embedding,auc,auprc,F1
0,In-Hospital Mortality,GRU,,2.54,3.11,1.64
1,In-Hospital Mortality,Average Multimodal,word2vec,1.84,4.69,1.82
2,In-Hospital Mortality,Average Multimodal,concat,1.79,4.40,2.60
3,In-Hospital Mortality,Average Multimodal,fasttext,1.65,3.14,1.34
4,,,,,,
5,In-Hospital Mortality,Average Multimodal,clinicalBERT,-,-,-
6,In-ICU Mortality,Average Multimodal,clinicalBERT,-,-,-
7,In-ICU Mortality,Average Multimodal,fasttext,1.75,3.69,1.98
8,In-ICU Mortality,Average Multimodal,word2vec,2.00,5.08,3.83
9,,,,,,


## Table 4

### Determine best model from table 3

In [14]:
tasks = ["los_3","los_7","mort_hosp", "mort_icu" ]

best_baseline = pd.concat([results_07, results_08]).fillna('-')

best_baseline = best_baseline.groupby(by=['task', 'embedding']).agg({"auc":['mean', 'std'], 
                                                                     "auprc":['mean', 'std'],
                                                                     "F1":['mean', 'std']})
best_baseline.columns = best_baseline.columns.map('_'.join).str.strip('_')
best_baseline.reset_index(inplace=True)

best_baseline_auc = best_baseline.loc[best_baseline.groupby("task")["auc_mean"].idxmax()]
best_baseline_auc = best_baseline_auc[["task", "auc_mean", 'auc_std']]

best_baseline_auprc = best_baseline.loc[best_baseline.groupby("task")["auprc_mean"].idxmax()]
best_baseline_auprc = best_baseline_auprc[["task", "auprc_mean", 'auprc_std']]

best_baseline_F1 = best_baseline.loc[best_baseline.groupby("task")["F1_mean"].idxmax()]
best_baseline_F1 = best_baseline_F1[["task", "F1_mean", 'F1_std']]

best_baseline = best_baseline_auc.merge(best_baseline_auprc, on='task').merge(best_baseline_F1, on='task')


# Get values in terms of %
for col in ['auc_mean', 'auc_std', 'auprc_mean', 'auprc_std', 'F1_mean', 'F1_std']:
   best_baseline[col] = best_baseline[col] * 100

# Rename the tasks
task_mapping = {
    "los_3": "LOS > 3 Days",
    "los_7": "LOS > 7 Days",
    "mort_hosp": "In-Hospital Mortality",
    "mort_icu": "In-ICU Mortality"
}
for k, v in task_mapping.items():
    best_baseline.loc[best_baseline["task"] == k, "task"] = v

# Format the numeric results
for metric in ["auc", "auprc", "F1"]:
    best_baseline[metric] = best_baseline[[f"{metric}_mean", f"{metric}_std"]].apply(lambda x: "{:.2f} +/- {:.3f}".format(x[0], x[1]), axis=1)

best_baseline["model_category"] = 'Best Baseline'
best_baseline["embedding"] = '-'

# Order the results in the manner desired
best_baseline = best_baseline[["task", "model_category", 'embedding', "auc", "auprc", "F1"]]

best_baseline

,task,model_category,embedding,auc,auprc,F1
0,LOS > 3 Days,Best Baseline,-,70.49 +/- 0.184,64.20 +/- 0.221,56.04 +/- 0.753
1,LOS > 7 Days,Best Baseline,-,73.74 +/- 0.219,23.36 +/- 0.222,4.58 +/- 1.901
2,In-Hospital Mortality,Best Baseline,-,88.35 +/- 0.104,58.91 +/- 0.679,48.02 +/- 1.286
3,In-ICU Mortality,Best Baseline,-,89.17 +/- 0.176,53.55 +/- 0.414,46.13 +/- 1.631


### Add best baseline to CNN results

In [15]:
def table_4(df1, df2):

    table_3_df = pd.concat([df1, df2])
#     table_3_df = table_3_df.drop(columns=['layer']).fillna('')
    table_3_df = table_3_df[["task", "model_category",'embedding', "auc", "auprc", "F1"]]
    table_3_df.sort_values(by='task', inplace=True, ascending=True)
    table_3_df = add_blank_rows_df(table_3_df, 4)
    return table_3_df

summary_results_paper_2 = table_4(best_baseline, summary_results_09)
summary_results_paper_2

,task,model_category,embedding,auc,auprc,F1
0,In-Hospital Mortality,Best Baseline,-,88.35 +/- 0.104,58.91 +/- 0.679,48.02 +/- 1.286
1,In-Hospital Mortality,Proposed Model,word2vec,88.00 +/- 0.187,58.06 +/- 0.559,45.53 +/- 1.879
2,In-Hospital Mortality,Proposed Model,fasttext,87.89 +/- 0.329,57.24 +/- 0.646,44.49 +/- 1.630
3,In-Hospital Mortality,Proposed Model,concat,88.14 +/- 0.170,57.68 +/- 0.386,45.69 +/- 1.620
4,,,,,,
5,In-ICU Mortality,Best Baseline,-,89.17 +/- 0.176,53.55 +/- 0.414,46.13 +/- 1.631
6,In-ICU Mortality,Proposed Model,word2vec,88.70 +/- 0.360,52.18 +/- 0.687,43.09 +/- 2.212
7,In-ICU Mortality,Proposed Model,fasttext,88.54 +/- 0.253,51.71 +/- 0.353,43.29 +/- 1.560
8,In-ICU Mortality,Proposed Model,concat,88.72 +/- 0.192,52.19 +/- 0.753,42.59 +/- 2.028
9,,,,,,


In [16]:
# results_difference_paper_2 = table_4(results_difference_07, results_difference_08)
# results_difference_paper_2

In [17]:
print(summary_results_paper_1.style.to_latex())

\begin{tabular}{lllllll}
 & task & model_category & embedding & auc & auprc & F1 \\
0 & In-Hospital Mortality & GRU &  & 87.58 +/- 0.288 & 55.26 +/- 0.975 & 43.93 +/- 2.186 \\
1 & In-Hospital Mortality & Average Multimodal & clinicalBERT & 88.35 +/- 0.104 & 58.10 +/- 0.404 & 44.51 +/- 1.519 \\
2 & In-Hospital Mortality & Average Multimodal & word2vec & 88.26 +/- 0.289 & 58.91 +/- 0.679 & 47.24 +/- 2.245 \\
3 & In-Hospital Mortality & Average Multimodal & concat & 88.21 +/- 0.195 & 58.62 +/- 0.519 & 48.02 +/- 1.286 \\
4 &  &  &  &  &  &  \\
5 & In-Hospital Mortality & Average Multimodal & fasttext & 87.74 +/- 0.323 & 57.61 +/- 0.451 & 46.84 +/- 1.530 \\
6 & In-ICU Mortality & Average Multimodal & fasttext & 88.89 +/- 0.278 & 52.05 +/- 0.613 & 44.89 +/- 1.829 \\
7 & In-ICU Mortality & Average Multimodal & concat & 89.05 +/- 0.252 & 53.16 +/- 0.372 & 45.48 +/- 1.420 \\
8 & In-ICU Mortality & Average Multimodal & word2vec & 89.17 +/- 0.176 & 53.55 +/- 0.414 & 46.13 +/- 1.631 \\
9 &  &  &  

In [18]:
print(summary_results_paper_2.style.to_latex())

\begin{tabular}{lllllll}
 & task & model_category & embedding & auc & auprc & F1 \\
0 & In-Hospital Mortality & Best Baseline & - & 88.35 +/- 0.104 & 58.91 +/- 0.679 & 48.02 +/- 1.286 \\
1 & In-Hospital Mortality & Proposed Model & word2vec & 88.00 +/- 0.187 & 58.06 +/- 0.559 & 45.53 +/- 1.879 \\
2 & In-Hospital Mortality & Proposed Model & fasttext & 87.89 +/- 0.329 & 57.24 +/- 0.646 & 44.49 +/- 1.630 \\
3 & In-Hospital Mortality & Proposed Model & concat & 88.14 +/- 0.170 & 57.68 +/- 0.386 & 45.69 +/- 1.620 \\
4 &  &  &  &  &  &  \\
5 & In-ICU Mortality & Best Baseline & - & 89.17 +/- 0.176 & 53.55 +/- 0.414 & 46.13 +/- 1.631 \\
6 & In-ICU Mortality & Proposed Model & word2vec & 88.70 +/- 0.360 & 52.18 +/- 0.687 & 43.09 +/- 2.212 \\
7 & In-ICU Mortality & Proposed Model & fasttext & 88.54 +/- 0.253 & 51.71 +/- 0.353 & 43.29 +/- 1.560 \\
8 & In-ICU Mortality & Proposed Model & concat & 88.72 +/- 0.192 & 52.19 +/- 0.753 & 42.59 +/- 2.028 \\
9 &  &  &  &  &  &  \\
10 & LOS > 3 Days & Be